In [6]:
# Loading Dataset
import utils

folder_path = 'datasets/cot-fsopt'
dataset = utils.load_dataset(folder_path)
dataset

,inputs,targets,_template_idx,_task_source,_task_name,_template_type
0,The man be showing his toys to adults and not ...,"Premise: ""Woman skates in possession of puck.""...",1,CoT,cot_esnli_ii,fs_opt
1,"[QUESTION] If ""A brown dog is on the ground gr...",A man is break dancing and collecting a lot of...,0,CoT,cot_esnli,fs_opt
2,Jax: Which of the following sentences is nonse...,Chain of thought: A watch is not worn on one's...,1,CoT,cot_sensemaking,fs_opt
3,"Student asked: Given the sentence ""A boy weari...",Let's think. A group is on the river regardles...,7,CoT,cot_esnli,fs_opt
4,**Q**\nIs the following sentence factually cor...,no\nLaughter is not a response of fear but to ...,1,CoT,cot_creak,fs_opt
...,...,...,...,...,...,...
95565,"Given the sentence ""A low wave is forming in t...",The ocean would not be located in the same pla...,1,CoT,cot_esnli,zs_opt
95566,Consider the question. How much 60% of 50 is g...,(60/100) * 50 – (50/100) * 30\n30 - 15 = 15,11,CoT,stream_aqua_ii,zs_opt
95567,Use reasoning to lead to the answer of the fol...,Car runs at speed that saves time of travellin...,5,CoT,cot_ecqa,zs_opt
95568,Which of these sentences doesn't make sense?\n...,Rats have sharp teeth that can penetrate most ...,7,CoT,cot_sensemaking,zs_opt


In [7]:
# Translate using unofficial API
from concurrent.futures import wait, FIRST_EXCEPTION
from termcolor import colored
from deep_translator.exceptions import RequestError
from errors import InvalidOutputError, MissingTranslationError, GeneralError, ReachedMaxRetriesError
import csv
import concurrent.futures
import utils
from multi_thread_handler import mth

dataset_name = 'cot-fsopt'
output_folder = f'outputs/{dataset_name}'
start_pointer_file_path = f'{output_folder}/start-pointer.txt'
next_file_index_file_path = f'{output_folder}/next-file-index.txt'
batch_size = 10

start_pointer = utils.read_integer_from_file(start_pointer_file_path)
next_file_index = utils.read_integer_from_file(next_file_index_file_path)
# start_pointer = 9461
# next_file_index = 9

file_name = utils.get_output_csv_path(output_folder, next_file_index, dataset_name, 'csv')
content_len = len(dataset)

error_occurred = False


def process_row(args):
    i, row = args
    input_text = row['inputs']
    target_text = row['targets']

    mth.safe_print(f"Processing Row: {i}")
    result = utils.choose_translation_method_and_translate(mth.rate_limited_translate, i, [input_text, target_text])
    if len(result) != 2:
        raise InvalidOutputError

    input_result = result[0]
    target_result = result[1]

    mth.safe_print(f"Queued Translation: {i}")
    return i, input_text, input_result, target_text, target_result


def translate_dataset(block_after: int = None):
    global error_occurred

    with open(file_name, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Id', 'Original Input', 'Translated Input', 'Original Target', 'Translated Target'])

        utils.update_integer_in_file(next_file_index_file_path, next_file_index + 1)

        end_pointer = start_pointer + block_after if block_after is not None else content_len
        current_batch_start = start_pointer
        connection_retries = 0
        start_time = utils.get_current_time()

        while current_batch_start < end_pointer and connection_retries < 3:
            current_batch_end = min(current_batch_start + batch_size, end_pointer)

            error_occurred = False

            with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
                futures = {executor.submit(process_row, (i, row)): i for i, row in
                           dataset.iloc[current_batch_start:current_batch_end].iterrows()}
                results = {}

                done, not_done = wait(futures.keys(), return_when=FIRST_EXCEPTION)
                non_network_error_occurred = False

                for future in done:
                    try:
                        i, input_text, input_result, target_text, target_result = future.result()
                        results[i] = (input_text, input_result, target_text, target_result)
                    except RequestError as e:
                        mth.safe_print(colored(f"[Network Error - Automatic Retry]: {e}", 'red'))
                        connection_retries += 1
                        if connection_retries >= 3:
                            raise ReachedMaxRetriesError
                        error_occurred = True
                        break
                    except Exception as e:
                        mth.safe_print(colored(f"[Non-Network Error]: {e}", 'red'))
                        error_occurred = True
                        non_network_error_occurred = True
                        raise e

                if error_occurred:
                    if current_batch_start == start_pointer:
                        mth.safe_print(colored("No batches could be processed. Exiting...", 'red'))
                        utils.update_integer_in_file(next_file_index_file_path, next_file_index)

                    for future in not_done:
                        mth.safe_print(colored(f"Cancelling unsubmitted futures: {future}", 'yellow'))
                        future.cancel()

                    if non_network_error_occurred:
                        break
                    else:
                        continue

                # Write the results of this batch
                for i in range(current_batch_start, current_batch_end):
                    print(f"Writing row {i}")
                    if i in results:
                        writer.writerow([i] + list(results[i]))
                    else:
                        raise MissingTranslationError(i)

                utils.update_integer_in_file(start_pointer_file_path, current_batch_end)
                file.flush()

                current_batch_start = current_batch_end
                current_time = utils.get_current_time()
                speed = utils.get_speed(current_batch_end - start_pointer, start_time, current_time)
                estimated_time = utils.get_estimated_time(content_len - start_pointer,
                                                          i - start_pointer, start_time,
                                                          current_time)

                mth.safe_print(
                    colored(
                        f"Moving to next batch. Translated {current_batch_end} of {content_len}, Elapsed (Secs): {current_time - start_time}, Estimated (Hrs): {estimated_time}, Speed: {speed}",
                        "green"))


translate_dataset()

Processing Row: 0
Processing Row: 1
Processing Row: 2
Processing Row: 3
Translated by blob for index 1, Time: 1722239865
Queued Translation: 1
Translated by blob for index 0, Time: 1722239865
Translated by blob for index 3, Time: 1722239865
Translated by blob for index 2, Time: 1722239865
Processing Row: 4
Queued Translation: 0
Queued Translation: 3
Queued Translation: 2
Processing Row: 5
Processing Row: 6
Processing Row: 7
Translated by blob for index 4, Time: 1722239865
Queued Translation: 4
Processing Row: 8
Translated by blob for index 5, Time: 1722239866
Queued Translation: 5
Processing Row: 9
Translated by blob for index 8, Time: 1722239866
Queued Translation: 8
Translated by blob for index 6, Time: 1722239866
Queued Translation: 6
Translated by blob for index 7, Time: 1722239866
Queued Translation: 7
Translated by blob for index 9, Time: 1722239866
Queued Translation: 9
Writing row 0
Writing row 1
Writing row 2
Writing row 3
Writing row 4
Writing row 5
Writing row 6
Writing row 

KeyboardInterrupt: 